In [1]:
import pandas as pd

In [2]:
train = pd.read_csv("datasets/final_datasets/pollution_train_2012_2020_scaled_interactions.csv")
test  = pd.read_csv("datasets/final_datasets/pollution_test_2021_2023_scaled_interactions.csv")

In [4]:

core_features = [
    "traffic_daily_total",
    "temperature_2m_max",
    "wind_speed_10m_max_(km/h)",
    "precipitation_sum_(mm)",
    "relative_humidity_2m_max_(%)",
]

extra_features = [
    "daylight_duration_(s)",
    "sunshine_duration_(s)",
    "rain_sum_(mm)",
    "snowfall_sum_(cm)",
    "wind_direction_10m_dominant_(°)",
    "et0_fao_evapotranspiration_(mm)",
    "shortwave_radiation_sum_(mj/m²)",
    "dew_point_2m_max",
    "cloud_cover_max_(%)",
    "cloud_cover_mean_(%)",
    "pressure_msl_max_(hpa)",
    "surface_pressure_max_(hpa)",
    "vapour_pressure_deficit_max_(kpa)",
]
base_features_no_conf = core_features + extra_features
interaction_features = [
    "int_traffic_temp",
    "int_traffic_wind",
    "int_traffic_precip",
    "int_traffic_humidity",
    "none"
]

## MLR: NO2

In [6]:
from sklearn.linear_model import LinearRegression

target = "NO2"
feat_list = base_features_no_conf + ["int_traffic_humidity"]

X_train = train[feat_list]
y_train = train[target]
X_test  = test[feat_list]
y_test  = test[target]

model_interaction = LinearRegression()
model_interaction.fit(X_train, y_train)
y_pred_interaction = model_interaction.predict(X_test)

X_train = train[base_features_no_conf]
y_train = train[target]
X_test  = test[base_features_no_conf]

model = LinearRegression()
model.fit(X_train, y_train)
y_pred_no_interaction = model.predict(X_test)

In [8]:
import numpy as np
from scipy.stats import ttest_rel

# y_true: true test labels
# y_pred_with: predictions with interaction
# y_pred_without: predictions without interaction
def ttest(y_test, y_pred_interaction, y_pred_no_interaction):
    err_with = (y_test - y_pred_interaction) ** 2
    err_without = (y_test - y_pred_no_interaction) ** 2

    t_stat, p_value = ttest_rel(err_without, err_with)

    print(f"t = {t_stat:.3f}, p = {p_value:.4g}")

In [9]:
ttest(y_test, y_pred_interaction, y_pred_no_interaction)

t = 0.327, p = 0.7436


### MLR: PM2.5

In [10]:
target = "PM2.5"
feat_list = base_features_no_conf + ["int_traffic_temp"]

X_train = train[feat_list]
y_train = train[target]
X_test  = test[feat_list]
y_test  = test[target]

model_interaction = LinearRegression()
model_interaction.fit(X_train, y_train)
y_pred_interaction = model_interaction.predict(X_test)

X_train = train[base_features_no_conf]
y_train = train[target]
X_test  = test[base_features_no_conf]

model = LinearRegression()
model.fit(X_train, y_train)
y_pred_no_interaction = model.predict(X_test)

In [11]:
ttest(y_test, y_pred_interaction, y_pred_no_interaction)

t = 0.953, p = 0.3409


### RF + O3

In [13]:
from sklearn.ensemble import RandomForestRegressor

target = "Ozone"
feat_list = base_features_no_conf + ["int_traffic_wind"]

X_train = train[feat_list]
y_train = train[target]
X_test  = test[feat_list]
y_test  = test[target]

rf = RandomForestRegressor(
        n_estimators=300,
        max_depth=50,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=0,
        n_jobs=-1,
    )

rf.fit(X_train, y_train)

y_pred_with_interaction = rf.predict(X_test)

X_train = train[base_features_no_conf]
y_train = train[target]
X_test  = test[base_features_no_conf]

rf_no_interaction = RandomForestRegressor(
        n_estimators=300,
        max_depth=20,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=0,
        n_jobs=-1,
    )

rf_no_interaction.fit(X_train, y_train)
y_pred_no_interaction = rf_no_interaction.predict(X_test)

In [14]:
ttest(y_test, y_pred_with_interaction, y_pred_no_interaction)

t = 1.609, p = 0.1083


### RF + CO

In [ ]:

target = "CO"
feat_list = base_features_no_conf + ["int_traffic_wind"]

X_train = train[feat_list]
y_train = train[target]
X_test  = test[feat_list]
y_test  = test[target]

rf = RandomForestRegressor(
        n_estimators=300,
        max_depth=5,
        min_samples_split=2,
        min_samples_leaf=2,
        random_state=0,
        n_jobs=-1,
    )

rf.fit(X_train, y_train)

y_pred_with_interaction = rf.predict(X_test)

X_train = train[base_features_no_conf]
y_train = train[target]
X_test  = test[base_features_no_conf]

rf_no_interaction = RandomForestRegressor(
        n_estimators=300,
        max_depth=20,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=0,
        n_jobs=-1,
    )

rf_no_interaction.fit(X_train, y_train)
y_pred_no_interaction = rf_no_interaction.predict(X_test)